In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install pyroomacoustics
!pip3 install torchmetrics


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 3.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import os
import sys
import glob
import random
import torchaudio
from tqdm import tqdm
import argparse


In [ ]:
sys.path.append('/content/drive/MyDrive/ColabNotebooks/IsoNet/')
#sys.path.append('/content/drive/MyDrive/ColabNotebooks/IsoNet/dataloader.py')
#sys.path.append('/content/drive/MyDrive/ColabNotebooks/IsoNet/training.py')
#sys.path.append('/content/drive/MyDrive/ColabNotebooks/IsoNet/tools.py')
#sys.path.append('/content/drive/MyDrive/ColabNotebooks/IsoNet/roomacoustics.py')
#sys.path.append('/content/drive/MyDrive/ColabNotebooks/IsoNet/metafiles.py')
#sys.path.append('/content/drive/MyDrive/ColabNotebooks/IsoNet/model.py')
root="/content/drive/MyDrive/ColabNotebooks/IsoNet/"


In [ ]:
from tools import splitStrings

def walk_through_files(path):
    test = os.path.join(path, "**" , '*.flac')
    return_list = glob.glob(test, recursive=True)

    list_possible_audio_files = []
    for path in return_list:
        minimum_file_length = 96000
        audiofile, sample_rate = torchaudio.load(path)

        if len(audiofile.T) >= minimum_file_length:
            list_possible_audio_files.append(path)

    #random_sample = random.choice(list_possible_audio_files)

    return list_possible_audio_files


def generate_meta_files(root, database_length, overwrite):

    root_dir = os.path.join(root, "dev-clean")
    root_dir_list_files = os.path.join(root, '**', '*.flac')
    print(root_dir)

    speaker_files = 'AudioFiles.txt'
    if not os.path.exists('AudioFiles.txt') or overwrite:
        files = glob.glob(root_dir_list_files, recursive=True)
        print("Number of files found: {}".format(len(files)))
        with open(speaker_files, 'w') as fp:
            for item in files:
                # write each item on a new line
                fp.write("%s\n" % item)
            print('List of All Audio Files Saved')
    else:
        with open(speaker_files) as f:
            files = f.read().splitlines()

    with open(os.path.join(root, "SPEAKERS.txt")) as f:
        speakers = f.read().splitlines()

    list_female = []
    list_male = []
    for speaker in speakers:
        if 'dev-clean' in speaker:
            res = splitStrings(speaker, '|')
            if res[1] == ' F ':
                list_female.append(res[0])
            else:
                list_male.append(res[0])

    if not os.path.exists(os.path.join(root, 'PossibleFemaleSpeakers.txt')) or overwrite:
        all_files_female = []
        for m in tqdm(range(len(list_male))):
            possible_file_female = walk_through_files(os.path.join(root_dir, list_female[m].strip()))
            all_files_female = all_files_female + possible_file_female

        with open(os.path.join(root, 'PossibleFemaleSpeakers.txt'), 'w') as fp:
            for item in all_files_female:
                # write each item on a new line
                fp.write("%s\n" % item)
            print('List of Female Speakers Saved')
    else:
        with open(os.path.join(root, 'PossibleFemaleSpeakers.txt')) as f:
            all_files_female = f.read().splitlines()

    if not os.path.exists(os.path.join(root, 'PossibleMaleSpeakers.txt')) or overwrite:
        all_files_male = []
        for m in tqdm(range(len(list_male))):
            possible_file_male = walk_through_files(os.path.join(root_dir, list_male[m].strip()))
            all_files_male = all_files_male + possible_file_male
        with open(os.path.join(root, 'PossibleMaleSpeakers.txt'), 'w') as fp:
            for item in all_files_male:
                # write each item on a new line
                fp.write("%s\n" % item)
            print('List of Male Speakers Saved')
    else:
        with open(os.path.join(root, 'PossibleMaleSpeakers.txt')) as f:
            all_files_male = f.read().splitlines()

    # Get Distribution
    print("Number of available male files: {}".format(len(all_files_male)))
    print("Number of available female files: {}".format(len(all_files_female)))

    target_all = random.choices(all_files_male, k=database_length//4)
    interferer_all = random.choices(all_files_male, k=database_length//4)

    target = random.choices(all_files_female, k=database_length//4)
    interferer = random.choices(all_files_male, k=database_length//4)

    target_all = target_all + target
    interferer_all = interferer_all + interferer

    target = random.choices(all_files_male, k=database_length//4)
    interferer = random.choices(all_files_female, k=database_length//4)

    target_all = target_all + target
    interferer_all = interferer_all + interferer

    target = random.choices(all_files_female, k=database_length//4)
    interferer = random.choices(all_files_female, k=database_length//4)

    target_all = target_all + target
    interferer_all = interferer_all + interferer

    with open(os.path.join(root, "training_list.txt"), 'w') as fp:
        for i in range(len(target_all)):
            # write each item on a new line
            fp.write(target_all[i] + ' ' + interferer_all[i] + '\n')
        print('Training List Saved and Finished')


generate_meta_files(root, database_length=20000, overwrite=False)


In [ ]:
import os
import glob
import sys
import random
import torchaudio
import numpy as np
from tqdm import tqdm

import torch
import torchmetrics

from torch.utils.data import DataLoader
from training import train, model_eval

from model import ConvTasNet
from dataloader import LibriSpeech
import argparse

torch.backends.cudnn.benchmark = True
mode = "train"
model_name = 'IsoNet.pt'

BATCH_SIZE = 16

EPOCHS = 100
addnoise = False
fs = 16000

total_length = 4
input_length = 16000 # in samples
win_length = int(fs*total_length)

with open(os.path.join(root, 'training_list.txt')) as f:
    training_files = f.read().splitlines()

print(mode)
if mode == 'train':
    print("-----------------Training Mode Started-----------------")
    print("-----------------Building Database-----------------")

    train_dataset = LibriSpeech(training_files, root_dir=root, fs=fs, length=total_length, length_input=input_length, batchsize=1)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, prefetch_factor=8, pin_memory=True)

    print("-----------------Loading Model-----------------")
    model = ConvTasNet()
    model.cuda()

    #criterion = torchmetrics.ScaleInvariantSignalNoiseRatio(full_state_update=False).cuda()

    criterion = torch.nn.MSELoss()

    optim_params = model.parameters()
    optimizer = torch.optim.Adam(optim_params, 0.001)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True, min_lr=0)
    
    eval_dataset = LibriSpeech(training_files[:100], root_dir=root, fs=fs, length=total_length, length_input=input_length, batchsize=1)
    eval_loader = DataLoader(eval_dataset, batch_size=1, num_workers=2, prefetch_factor=8, pin_memory=True)
    print("-----------------Starting Training------------------")
    for epoch in range(EPOCHS):
        

        model = train(train_loader, model, criterion, optimizer, epoch, scheduler, loss_mode="sisnr")

        torch.save(model, model_name)

        print("\n -----------------Starting Evaluation----------------")
        
        isolated_predictions = model_eval(eval_loader, model, fs, total_length)
else:
    print("-----------------Evaluation Mode Started-----------------")

    leaked_target, leaked_interferer, target_audio = next(iter(eval_loader))

    target_mix = torch.reshape(torch.squeeze(leaked_target), shape=(1, int(fs*total_length)))
    interfering_mix = torch.reshape(torch.squeeze(leaked_interferer), shape=(1, int(fs*total_length)))
    target = torch.reshape(torch.squeeze(target_audio), shape=(1, int(fs*total_length)))

    for k in range(len(target_mix)):
        torchaudio.save('./evaluation_sample/target_mix{}.wav'.format(k),
                        target_mix, 16000)
        torchaudio.save('./evaluation_sample/target{}.wav'.format(k),
                        target, 16000)
        torchaudio.save('./evaluation_sample/interferer_mix{}.wav'.format(k),
                        interfering_mix, 16000)
    print("-----------------Loading Model---------------------")
    loaded_model = torch.load(model_name).cuda()

    print("-----------------Loading Completed, starting evaluation...-------------")
    isolated_predictions = model_eval(eval_loader, loaded_model, fs, total_length)
    print("Files Successfully Written, Finished")
